In [1]:
import sys

In [2]:
sys.path.append('..')

In [3]:
from jamo import j2hcj, h2j
import pandas as pd
import numpy as np
from konlpy.tag import Twitter

from sklearn.metrics import accuracy_score, f1_score
twitter = Twitter()

/Users/1003874/anaconda/envs/lstm/lib/python3.6/site-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [4]:
from tensorflow.python.ops.rnn import bidirectional_dynamic_rnn as bi_rnn
from tensorflow.contrib.rnn import BasicLSTMCell
from utils.prepare_data import *
import time
from utils.model_helper import *

/Users/1003874/anaconda/envs/lstm/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [5]:
def testcase_shuffle_data_load():
    df = pd.read_csv('/Users/1003874/bsdev/Text-classification-with-CNN-RNN-with-Tensorflow/Ch01_Data_load/data/preprocessed_mart_digi_speech_act.csv')
    df['clean_text'] = df['clean_text'].apply(lambda x:x.lower())
    s = df['class'].value_counts()
    cols = s[s>50].index.tolist()

    sub_list = []
    for col in cols:
        sub_list.append(df[df['class'] == col])
    df = pd.concat(sub_list, axis=0)
    df = df.sample(len(df), random_state=10).reset_index(drop=True)

    added_df = pd.read_csv('/Users/1003874/bsdev/Text-classification-with-CNN-RNN-with-Tensorflow/Ch01_Data_load/data/added_dataset.csv',sep='\t')
    added_df['clean_text'] = added_df['sent'].apply(lambda x:x.lower())

    # class 제거
    added_df = added_df[added_df['class'].apply(lambda x: True if x in cols else False)]
    added_df.info()

    # duplicated sent 제거
    added_df = added_df[added_df['clean_text'].apply(lambda x: False if x in df['clean_text'].tolist() else True)]
    added_df.info()

    # train_df merge

    size = len(df)
    rate = 0.2
    train_df = df.iloc[:int(size * (1 - rate))]
    test_df = df.iloc[int(size * (1 - rate)):]

    all_df = pd.concat([train_df[['clean_text', 'class']],
                        test_df[['clean_text', 'class']],
                        added_df[['clean_text', 'class']]])

    all_df = all_df.sample(len(all_df), random_state=10)
    all_df.reset_index(drop=True, inplace=True)

    label_df = pd.get_dummies(all_df['class'])

    train_df = all_df.iloc[1062:]
    test_df = all_df.iloc[:1062]

    cols = label_df.columns

    print(len(train_df), len(test_df))

    TRAIN_DOC, TRAIN_LABEL, TEST_DOC, TEST_LABEL = \
        train_df['clean_text'].values, label_df.loc[train_df.index].values, test_df['clean_text'].values, label_df.loc[
            test_df.index].values

    return TRAIN_DOC, TRAIN_LABEL, TEST_DOC, TEST_LABEL, cols

In [6]:
def testcase_add_data_load():
    df = pd.read_csv('/Users/1003874/bsdev/Text-classification-with-CNN-RNN-with-Tensorflow/Ch01_Data_load/data/preprocessed_mart_digi_speech_act.csv')
    df['clean_text'] = df['clean_text'].apply(lambda x:x.lower())
    s = df['class'].value_counts()
    cols = s[s>50].index.tolist()

    sub_list = []
    for col in cols:
        sub_list.append(df[df['class'] == col])
    df = pd.concat(sub_list, axis=0)
    df = df.sample(len(df), random_state=10).reset_index(drop=True)

    added_df = pd.read_csv(
        '/Users/1003874/bsdev/Text-classification-with-CNN-RNN-with-Tensorflow/Ch01_Data_load/data/added_dataset.csv',
        sep='\t')
    added_df['clean_text'] = added_df['sent'].apply(lambda x:x.lower())

    # class 제거
    added_df = added_df[added_df['class'].apply(lambda x: True if x in cols else False)]
    added_df.info()

    # duplicated sent 제거
    added_df = added_df[added_df['clean_text'].apply(lambda x: False if x in df['clean_text'].tolist() else True)]
    added_df.info()

    # train_df merge

    size = len(df)
    rate = 0.2
    train_df = df.iloc[:int(size * (1 - rate))]
    test_df = df.iloc[int(size * (1 - rate)):]
    train_df['setlabel'] = 1
    test_df['setlabel'] = 0
    added_df['setlabel'] = 2

    all_df = pd.concat([train_df[['clean_text', 'class', 'setlabel']],
                        test_df[['clean_text', 'class', 'setlabel']],
                        added_df[['clean_text', 'class','setlabel']]])

    all_df.reset_index(drop=True, inplace=True)

    label_df = pd.get_dummies(all_df['class'])

    train_df = all_df[all_df['setlabel'] != 0]
    test_df = all_df[all_df['setlabel'] == 0]

    cols = label_df.columns

    print(len(train_df), len(test_df))

    TRAIN_DOC, TRAIN_LABEL, TEST_DOC, TEST_LABEL = \
        train_df['clean_text'].values, label_df.loc[train_df.index].values, test_df['clean_text'].values, label_df.loc[
            test_df.index].values

    return TRAIN_DOC, TRAIN_LABEL, TEST_DOC, TEST_LABEL, cols

In [7]:
def digi_data_load():
    df = pd.read_csv('/Users/1003874/bsdev/Text-classification-with-CNN-RNN-with-Tensorflow/Ch01_Data_load/data/preprocessed_mart_digi_speech_act.csv')
    df['clean_text'] = df['clean_text'].apply(lambda x:x.lower())
    s = df['class'].value_counts()
    cols = s[s>50].index.tolist()

    sub_list = []
    for col in cols:
        sub_list.append(df[df['class'] == col])
    df = pd.concat(sub_list, axis=0)
    df = df.sample(len(df), random_state=10).reset_index(drop=True)

    size = len(df)
    rate = 0.2
    train_df = df.iloc[:int(size*(1-rate))]
    test_df = df.iloc[int(size*(1-rate)):]
    label_df = pd.get_dummies(df['class'])
    cols = label_df.columns
    print(len(train_df), len(test_df))

    TRAIN_DOC, TRAIN_LABEL, TEST_DOC, TEST_LABEL = \
        train_df['clean_text'].values, label_df.loc[train_df.index].values, test_df['clean_text'].values, label_df.loc[test_df.index].values

    return TRAIN_DOC, TRAIN_LABEL, TEST_DOC, TEST_LABEL, cols

In [8]:
class ABLSTM(object):
    def __init__(self, config):
        self.max_len = config["max_len"]
        self.hidden_size = config["hidden_size"]
        self.vocab_size = config["vocab_size"]
        self.embedding_size = config["embedding_size"]
        self.n_class = config["n_class"]
        self.learning_rate = config["learning_rate"]

        # placeholder
        self.x = tf.placeholder(tf.int32, [None, self.max_len])
        self.label = tf.placeholder(tf.int32, [None])
        self.keep_prob = tf.placeholder(tf.float32)

    def build_graph(self):
        print("building graph")
        # Word embedding
        embeddings_var = tf.Variable(tf.random_uniform([self.vocab_size, self.embedding_size], -1.0, 1.0),
                                     trainable=True)
        batch_embedded = tf.nn.embedding_lookup(embeddings_var, self.x)

        rnn_outputs, _ = bi_rnn(BasicLSTMCell(self.hidden_size),
                                BasicLSTMCell(self.hidden_size),
                                inputs=batch_embedded, dtype=tf.float32)

        fw_outputs, bw_outputs = rnn_outputs

        W = tf.Variable(tf.random_normal([self.hidden_size], stddev=0.1))
        H = fw_outputs + bw_outputs  # (batch_size, seq_len, HIDDEN_SIZE)
        M = tf.tanh(H)  # M = tanh(H)  (batch_size, seq_len, HIDDEN_SIZE)

        self.alpha = tf.nn.softmax(tf.reshape(tf.matmul(tf.reshape(M, [-1, self.hidden_size]),
                                                        tf.reshape(W, [-1, 1])),
                                              (-1, self.max_len)))  # batch_size x seq_len
        r = tf.matmul(tf.transpose(H, [0, 2, 1]),
                      tf.reshape(self.alpha, [-1, self.max_len, 1]))
        r = tf.squeeze(r)
        h_star = tf.tanh(r)  # (batch , HIDDEN_SIZE

        h_drop = tf.nn.dropout(h_star, self.keep_prob)

        # Fully connected layer（dense layer)
        FC_W = tf.Variable(tf.truncated_normal([self.hidden_size, self.n_class], stddev=0.1))
        FC_b = tf.Variable(tf.constant(0., shape=[self.n_class]))
        y_hat = tf.nn.xw_plus_b(h_drop, FC_W, FC_b)

        self.loss = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y_hat, labels=self.label))

        # prediction
        self.prediction = tf.argmax(tf.nn.softmax(y_hat), 1)

        # optimization
        loss_to_minimize = self.loss
        tvars = tf.trainable_variables()
        gradients = tf.gradients(loss_to_minimize, tvars, aggregation_method=tf.AggregationMethod.EXPERIMENTAL_TREE)
        grads, global_norm = tf.clip_by_global_norm(gradients, 1.0)

        self.global_step = tf.Variable(0, name="global_step", trainable=False)
        self.optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate)
        self.train_op = self.optimizer.apply_gradients(zip(grads, tvars), global_step=self.global_step,
                                                       name='train_step')
        print("graph built successfully!")


In [9]:
TRAIN_DOC, TRAIN_LABEL, TEST_DOC, TEST_LABEL, cols = digi_data_load()
#TRAIN_DOC, TRAIN_LABEL, TEST_DOC, TEST_LABEL, cols = testcase_add_data_load()
#TRAIN_DOC, TRAIN_LABEL, TEST_DOC, TEST_LABEL, cols = testcase_shuffle_data_load()

4244 1062


In [10]:
train_intent = np.apply_along_axis(np.argmax,1,TRAIN_LABEL)
test_intent = np.apply_along_axis(np.argmax,1,TEST_LABEL)

In [20]:
%%time
train_doc = TRAIN_DOC.tolist()
test_doc = TEST_DOC.tolist()

#train_doc = pd.Series(list(map(lambda x: ' '.join(twitter.morphs(x)), train_doc)))
#test_doc = pd.Series(list(map(lambda x: ' '.join(twitter.morphs(x)), test_doc)))
train_doc = pd.Series(list(map(lambda x: j2hcj(h2j(x)), train_doc)))\
    .apply(lambda x: ' '.join(list(x.replace(' ','ⓢ'))))
test_doc = pd.Series(list(map(lambda x: j2hcj(h2j(x)), test_doc)))\
    .apply(lambda x: ' '.join(list(x.replace(' ','ⓢ'))))

CPU times: user 417 ms, sys: 5.16 ms, total: 422 ms
Wall time: 421 ms


In [21]:
y_train = pd.Series(train_intent)
y_test = pd.Series(test_intent)

In [37]:
x_train, x_test, tokenizer = \
        data_preprocessing_v2(train_doc, test_doc, max_len=128,max_words=500)
vocab_size= tokenizer.num_words

In [38]:
tokenizer.word_docs

{'0': 1,
 '1': 84,
 '2': 3,
 '4': 2,
 '5': 2,
 'a': 82,
 'b': 1,
 'c': 6,
 'd': 4,
 'e': 6,
 'f': 5,
 'g': 1,
 'h': 1,
 'i': 38,
 'k': 31,
 'l': 4,
 'm': 8,
 'n': 4,
 'o': 20,
 'p': 57,
 'q': 7,
 'r': 4,
 's': 64,
 't': 25,
 'u': 3,
 'v': 27,
 'w': 1,
 'y': 25,
 'ⓢ': 3544,
 'ㄱ': 2808,
 'ㄲ': 234,
 'ㄴ': 3476,
 'ㄶ': 76,
 'ㄷ': 1966,
 'ㄸ': 325,
 'ㄹ': 2232,
 'ㄺ': 1,
 'ㅀ': 6,
 'ㅁ': 2268,
 'ㅂ': 1712,
 'ㅃ': 34,
 'ㅄ': 113,
 'ㅅ': 2218,
 'ㅆ': 628,
 'ㅇ': 3829,
 'ㅈ': 2046,
 'ㅉ': 51,
 'ㅊ': 682,
 'ㅋ': 607,
 'ㅌ': 516,
 'ㅍ': 1097,
 'ㅎ': 1788,
 'ㅏ': 3430,
 'ㅐ': 1322,
 'ㅑ': 152,
 'ㅒ': 6,
 'ㅓ': 2282,
 'ㅔ': 1571,
 'ㅕ': 906,
 'ㅖ': 95,
 'ㅗ': 2234,
 'ㅘ': 540,
 'ㅙ': 129,
 'ㅚ': 434,
 'ㅛ': 1610,
 'ㅜ': 2084,
 'ㅝ': 354,
 'ㅞ': 10,
 'ㅟ': 229,
 'ㅠ': 93,
 'ㅡ': 2082,
 'ㅢ': 191,
 'ㅣ': 2619}

In [39]:
x_dev = x_test
y_dev = y_test
dev_size = len(x_dev)
test_size = len(x_test)
print("Validation Size: ", dev_size)

config = {
    "max_len": 128,
    "hidden_size": 64,
    "vocab_size": vocab_size,
    "embedding_size": 64,
    "n_class": 33,
    "learning_rate": 1e-3,
    "batch_size": 4,
    "train_epoch": 20
}

classifier = ABLSTM(config)
classifier.build_graph()

Validation Size:  1062
building graph
graph built successfully!


In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
dev_batch = (x_dev, y_dev)
start = time.time()
for e in range(config["train_epoch"]):

    t0 = time.time()
    print("Epoch %d start !" % (e + 1))
    for x_batch, y_batch in fill_feed_dict(x_train, y_train, config["batch_size"]):
        return_dict = run_train_step(classifier, sess, (x_batch, y_batch))
        attn = get_attn_weight(classifier, sess, (x_batch, y_batch))
        # plot the attention weight
        # print(np.reshape(attn, (config["batch_size"], config["max_len"])))
    t1 = time.time()

    print("Train Epoch time:  %.3f s" % (t1 - t0))
    dev_acc = run_eval_step(classifier, sess, dev_batch)
    print("validation accuracy: %.3f " % dev_acc)

print("Training finished, time consumed : ", time.time() - start, " s")
print("Start evaluating:  \n")
cnt = 0
test_acc = 0
for x_batch, y_batch in fill_feed_dict(x_test, y_test, config["batch_size"]):
    acc = run_eval_step(classifier, sess, (x_batch, y_batch))
    test_acc += acc
    cnt += 1

print("Test accuracy : %f %%" % (test_acc / cnt * 100))


Epoch 1 start !
Train Epoch time:  60.779 s
validation accuracy: 0.284 
Epoch 2 start !
Train Epoch time:  63.751 s
validation accuracy: 0.407 
Epoch 3 start !
Train Epoch time:  63.705 s
validation accuracy: 0.458 
Epoch 4 start !
Train Epoch time:  68.619 s
validation accuracy: 0.519 
Epoch 5 start !
Train Epoch time:  66.110 s
validation accuracy: 0.526 
Epoch 6 start !
Train Epoch time:  61.886 s
validation accuracy: 0.546 
Epoch 7 start !
Train Epoch time:  67.526 s
validation accuracy: 0.568 
Epoch 8 start !
Train Epoch time:  66.166 s
validation accuracy: 0.574 
Epoch 9 start !
Train Epoch time:  63.638 s
validation accuracy: 0.589 
Epoch 10 start !
Train Epoch time:  64.058 s
validation accuracy: 0.570 
Epoch 11 start !
Train Epoch time:  63.254 s
validation accuracy: 0.603 
Epoch 12 start !
Train Epoch time:  61.218 s
validation accuracy: 0.609 
Epoch 13 start !
Train Epoch time:  63.576 s
validation accuracy: 0.615 
Epoch 14 start !
Train Epoch time:  65.016 s
validation accu

In [ ]:
preds = []
reals = []
attns = []
for x_batch, y_batch in fill_feed_dict(x_test, y_test,1062, isshuffle=False):
    pred = get_prediction(classifier, sess, (x_batch, y_batch))
    attn = get_attn_weight(classifier, sess, (x_batch, y_batch))
    preds += pred.tolist()
    attns += attn.tolist()
    reals += y_batch.tolist()

In [ ]:
test_df = pd.DataFrame({'sent':test_doc,'attn':attns, 'pred':preds, 'real':reals})
test_df

In [ ]:
test_df['real'] = test_df['real'].apply(lambda x: cols[x])
test_df['pred'] = test_df['pred'].apply(lambda x: cols[x])

In [ ]:
f1_score(test_df['real'],test_df['pred'],average='weighted')

In [ ]:
test_df[test_df['real'] == test_df['pred']]